In [ ]:
#《深入量化》 学习笔记
#课程地址：https://www.deeplearning.ai/short-courses/quantization-in-depth/
#量化是AI软件技术栈的关键部分，用于压缩LLM或其他模型，从头开始实现线性量化的最常见变体，即非对称式和对称式，等等量化方式

#在本课程中，我们将直接动手实现线性量化的不同变体，如上面说的的对称和非对称模式；还会在纯Pytorch 中实现不同的量化粒度，如逐张量、逐通道以及逐组量化，每个算法都有其优点和缺点。

#然后，我们会构建自己的量化器，使用逐通道量化方案，将任何模型量化为8位精度，这个方案可以应用于任何模型，不论其模态的性质，
#也就是说，可以将这个方法应用于文本、视觉、音频、甚至多模态模型。

#解决量化过程中的挑战：
##存储低比特精度权重（例如4bit或2bit），最常见的方式是权重打包，即将所有2或4比特张量打包到更大的8比特张量中，而无需分配额外的内存。

#最后将回顾目前最新技术方法，以便在Hugging Face生态系统内执行无性能下降量化。

#量化是当今大模型实际使用的一个非常重要的部分，深入了解它将帮助我们更有效地构建、部署和使用模型。


In [ ]:
'''
About this course
关于本课程

In Quantization in Depth you will build model quantization methods to shrink model weights to ¼ their original size,
and apply methods to maintain the compressed model’s performance. Your ability to quantize your models can make them more accessible, 
and also faster at inference time. 
在《深度量化》中，您将构建模型量化方法，将模型权重压缩至原来的四分之一，并应用方法保持压缩模型的性能。
您对模型进行量化的技能能够使其更易于获取，同时在推理时速度更快。

Implement and customize linear quantization from scratch so that you can study the tradeoff between space and performance, 
and then build a general-purpose quantizer in PyTorch that can quantize any open source model. 
You’ll implement  techniques to compress model weights from 32 bits to 8 bits and even 2 bits.
从零开始实现并自定义线性量化，以便研究空间与性能之间的权衡，然后在 PyTorch 中构建一个通用量化器，能够对任何开源模型进行量化。
您将实现将模型权重从 32 位压缩到 8 位甚至 2 位的技术。

Join this course to:
参加本课程可：

1.Build and customize linear quantization functions, choosing between two “modes”: asymmetric and symmetric;
and three granularities: per-tensor, per-channel, and per-group quantization.
构建并自定义线性量化函数，可在两种“模式”（非对称和对称）以及三种粒度（张量级、通道级和组级量化）之间进行选择；

2.Measure the quantization error of each of these options as you balance the performance and space tradeoffs for each option.
衡量每种选项的量化误差，以平衡每种选项的性能和空间权衡。

3.Build your own quantizer in PyTorch, to quantize any open source model’s dense layers from 32 bits to 8 bits.
在 PyTorch 中构建自己的量化器，将任何开源模型的密集层从 32 位量化到 8 位。

4. Go beyond 8 bits, and pack four 2-bit weights into one 8-bit integer.
突破 8 位，将四个 2 位权重打包到一个 8 位整数中。

Quantization in Depth lets you build and customize your own linear quantizer from scratch, going beyond standard open source libraries such as PyTorch and Quanto, 
which are covered in the short course Quantization Fundamentals, also by Hugging Face.
《深度量化》课程让您从零开始构建和自定义自己的线性量化器，超越了 PyTorch 和 Quanto 等标准开源库，这些内容在 Hugging Face 推出的短期课程《量化基础》中均有介绍。

This course gives you the foundation to study more advanced quantization methods, some of which are recommended at the end of the course.
本课程为您学习更高级的量化方法打下基础，其中一些方法会在课程结束时为您推荐。
'''

In [2]:
#关于上面第4点的解释与示例：将4个2位权重打包进一个8位整数中

##通常模型量化里，8 位整数（int8）是比较常用的量化格式。“Go beyond 8 bits” 意味着要使用比 8 位更低的比特数来表示模型的权重，像 4 位或者 2 位。
##使用更低比特数能够进一步减少模型的存储需求和计算量。
##把 4 个 2 位的权重打包进一个 8 位整数里。因为一个 2 位整数可以表示 \(2^2 = 4\) 种不同的数值，而一个 8 位整数可以表示 \(2^8 = 256\) 种不同的数值。
##所以从理论上来说，一个 8 位整数能够容纳 4 个 2 位整数的信息。

# 假设这是4个2位的权重
weights_2bit = [0b01, 0b10, 0b11, 0b00]

# 初始化一个8位整数
packed_8bit = 0

# 把4个2位权重打包进一个8位整数
for i, weight in enumerate(weights_2bit):
    packed_8bit |= weight << (i * 2)

print(f"Packed 8-bit integer: {bin(packed_8bit)}")

# 解包操作
unpacked_weights_2bit = []
for i in range(4):
    unpacked_weight = (packed_8bit >> (i * 2)) & 0b11
    unpacked_weights_2bit.append(unpacked_weight)

print(f"Unpacked 2-bit weights: {[bin(w) for w in unpacked_weights_2bit]}")

Packed 8-bit integer: 0b111001
Unpacked 2-bit weights: ['0b1', '0b10', '0b11', '0b0']


#overview

##量化方法被用来使模型变得更小，从而使其更易于被AI社区使用

##量化旨在以较低精度保存模型的参数（即模型压缩：Model compression），如下图




![量化示例](./img/00-overview.jpg)